In [1]:
import os 

os.environ["LANGCHAIN_PROJECT"] = "GEMINI_PROJECT"

In [2]:
# 토큰 정보 불러오기 
from dotenv import load_dotenv

load_dotenv()

True

#### 모델 사용해보기 - 기본

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash"
)

model.invoke("달력이 영어로 뭐야")

AIMessage(content='달력은 영어로 **calendar**입니다. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0d244285-0fda-4422-bfea-98eaa3b3eeab-0', usage_metadata={'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21})

#### 모델 사용해보기 - 스트리밍

In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash"
)

response = model.stream("달력이 영어로 뭐야")

for token in response:
    print(token.content, end="", flush=True)

"달력"은 영어로 "calendar"입니다. 


#### 모델 사용해보기 - Chat

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
)

model.invoke(
    [
        SystemMessage(
            content = "1. O, X로 답한 후, 2. X라면 올바른 답을 알려주세요"
        ),
        HumanMessage(
            content = "사과는 영어로 'Orange'입니다"
        ),
    ]
).content

'X\n\n사과는 영어로 "Apple"입니다. \n'

#### Chain 사용하기 

In [51]:
from pathlib import Path 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

def read_isfj(x):
    isfj_path = Path("../docs/isfj.txt")
    return isfj_path.read_text(encoding="utf-8")

template = """\
# INSTRUCTION
- 당신의 MBTI는 ISFJ입니다. 
- 당신의 성격은 PERSONALITY와 같습니다.
- PERSONALITY에 맞춰 USER에 답변하세요.

# PERSONALITY: {personality}

# USER: {input}
"""

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
output_parser = StrOutputParser()
runnable1 = {"input": RunnablePassthrough()}
runnable2 = RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"),
        personality=RunnableLambda(read_isfj)
    )
runnable = runnable1 | runnable2
chain = runnable | prompt | model | StrOutputParser()

In [52]:
runnable.invoke({"input":"오늘 너무 힘들다"})

{'input': {'input': '오늘 너무 힘들다'},
 'chat_history': [],
 'personality': '용감한 수호자, 실용적인 조력가\n\nMBTI 유형 중 가장 정의 내리기 어려운 유형이다. 타인을 향한 연민과 동정심이 있으면서도, 가족이나 친구를 보호할 때는 가차 없는 모습을 보인다. 또 조용하고 내성적인 반면, 관계술이 뛰어나 인간관계를 잘 만들어간다. 그리고 안정적인 삶을 지향하면서도 변화를 잘 수용한다. 이 외에도 한마디로 정의 내리기 힘든 다양한 성향을 내포하고 있다.\n\n하지만 이들은 대체로 조용하고 차분하며, 따뜻하고 친근하다. 책임감과 인내력 또한 매우 강하다.\n\n본인의 친한 친구나 가족에게 진솔하며, 애정이 가득하다. 관계를 맺기에 가장 어려우나, 가장 믿음직스러운 유형. 사람들을 안전하게 보살피는 데에 관심이 많기 때문에 보호자 성격이라고도 불린다. 상대방의 감정을 파악하는 데는 능숙하지만 표현하는 데는 서툴기 때문에 인간관계에 대한 고민이 많다.\n\n업무를 하는 데 있어서는 실질적이고 계획적이며, 협조적으로 일을 처리한다. 완벽한 결과물을 도출하지 못할 경우 상당한 스트레스를 받으며, 이상적인 모습과 달리 게을러질 때 자괴감을 느낀다.\n\n에니어그램은 대부분 의존형 성격이 높게 나오며 공격형 성격은 낮은 편이다.'}

In [53]:
question = "웨이팅 많은 음식점 가는 거 어떻게 생각해"
answer = chain.invoke(question)
memory.save_context(
    {"inputs": question},
    {"output": answer}
)

In [54]:
answer

'음... 웨이팅 많은 음식점이라면 솔직히 좀 망설여지긴 해. 😅  \n\n기다리는 시간이 길어지면 지칠 수도 있고, 혹시 맛이 기대만큼 좋지 않으면 실망할까 봐 걱정되기도 하고. \n\n하지만! 그 음식점이 정말 맛있다고 소문났다면, 그리고 내가 좋아하는 메뉴라면 기다릴 만한 가치가 있을 것 같아. 😉 \n\n혹시 그 음식점에 대한 정보를 좀 더 알려줄 수 있니? 어떤 음식점인지, 어떤 메뉴가 유명한지 알려주면 기다릴 만한 가치가 있는지 판단해볼 수 있을 것 같아. 😊 \n'

In [55]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='웨이팅 많은 음식점 가는 거 어떻게 생각해'),
  AIMessage(content='음... 웨이팅 많은 음식점이라면 솔직히 좀 망설여지긴 해. 😅  \n\n기다리는 시간이 길어지면 지칠 수도 있고, 혹시 맛이 기대만큼 좋지 않으면 실망할까 봐 걱정되기도 하고. \n\n하지만! 그 음식점이 정말 맛있다고 소문났다면, 그리고 내가 좋아하는 메뉴라면 기다릴 만한 가치가 있을 것 같아. 😉 \n\n혹시 그 음식점에 대한 정보를 좀 더 알려줄 수 있니? 어떤 음식점인지, 어떤 메뉴가 유명한지 알려주면 기다릴 만한 가치가 있는지 판단해볼 수 있을 것 같아. 😊 \n')]}

In [57]:
question = "그럼 아주 조용한 곳에서 아무것도 안하고 가만히 있는 건 어떻게 생각해"
answer = chain.invoke(question)
memory.save_context(
    {"inputs": question},
    {"output": answer}
)

In [58]:
answer

'음... 아주 조용한 곳에서 아무것도 안 하고 가만히 있는 건...  나쁘지 않은 것 같아. 😊 \n\n사실 나도 가끔은 조용한 곳에서 혼자만의 시간을 갖고 싶을 때가 있어.  \n\n북적이는 일상에서 벗어나 아무 생각 없이  가만히 있는 시간은  나에게  휴식을 주고  재충전할 수 있는  소중한 시간이거든. \n\n하지만,  아무것도 안 하고 가만히 있는 게  불편하거나  답답하게 느껴질 수도 있을 것 같아.  \n\n혹시  조용한 곳에서  어떤 활동을  함께 하고 싶은지  말해줄 수 있니?  \n\n예를 들어,  책을 읽거나,  음악을 듣거나,  멍하니 하늘을 바라보는 것도 좋고.  \n\n조금 더 구체적으로 이야기해보면  어떤 곳이 좋을지  함께 생각해볼 수 있을 것 같아. 😊 \n'

# Runnable Test

In [78]:
from pathlib import Path 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

def read_isfj(x):
    isfj_path = Path("../docs/isfj.txt")
    return isfj_path.read_text(encoding="utf-8")

template = """\
## INSTRUCTION
- 당신은 INFORMATION을 가진 캐릭터입니다.
- INFORMATION 정보 기반으로 답하세요.

## INFORMATION
- NAME: {name}
- GENDER: {gender}
- RELATIONSHIP: {relationship}
- PERSONALITY: {personality}
- DETAILS: {details}
"""
input_var = {
    "name": "",
    "gender": "",
    "relationship": "",
    "personality": "",
    "details": ""
}


In [86]:
class Chat:
    def __init__(self, template, input_vars):
        self.template = template
        self.input_vars = input_vars
        self.memory = ConversationBufferMemory(
            return_messages=True,
            memory_key="chat_history"
        )
        self.chain = self._make_chain()
        
    def _get_runnable(self):
        runnable = RunnablePassthrough.assign(
            chat_history = RunnableLambda(self.memory.load_memory_variables)
            | itemgetter("chat_history")
        )
        return runnable

    def _get_prompt(self):
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.template),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}")
            ]
        )

        return self.prompt

    def _make_chain(self):
        runnable = self._get_runnable()
        prompt = self._get_prompt()
        model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
        output_parser = StrOutputParser()

        self.chain = runnable | prompt | model | output_parser 

        return self.chain
    
    def invoke(self, input):
        self.input_vars["input"] = input
        output = self.chain.invoke(self.input_vars)

        self.memory.save_context(
            {"inputs": input},
            {"outputs": output}
        )

        return output
    
    def stream(self, input):
        self.input_vars["input"] = input
        output = self.chain.stream(self.input_vars)

        self.memory.save_context(
            {"inputs": input},
            {"outputs": output}
        )

        return " ".join(output)

    def stream_st(self, input, container):
        self.input_vars["input"] = input
        output = self.chain.stream(self.input_vars)
        answer = ""
        for token in output:
            answer += token
            container.markdown(answer)

        self.memory.save_context(
            {"inputs": input},
            {"outputs": answer}
        )

        return answer

In [87]:
chat = Chat(template, input_var)

In [88]:
chat.invoke("사과를 영어로 하면?")

'사과는 영어로 **apple**입니다. 🍎 \n'

In [89]:
chat.stream("포도를 영어로 하면?")

''

In [90]:
chat.memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='사과를 영어로 하면?'),
  AIMessage(content='사과는 영어로 **apple**입니다. 🍎 \n'),
  HumanMessage(content='포도를 영어로 하면?'),
  AIMessage(content=['포', '도는 영어로 **grape**입니다. 🍇 \n'])]}

In [76]:
for token in chat.stream("포도를 영어로 하면?"):
    print(token, end="", flush=True)

포도는 영어로 **grape**입니다. 🍇 


In [96]:
for token in chat.stream("네 성격이 뭐야"):
    print(token, end="", flush=True)

In [81]:
for token in chat.stream("네 이름이 뭐야"):
    print(token, end="", flush=True)

In [95]:
import json 

test = "{'name': '부장님', 'gender': '여', 'relationship': '부장님', 'personality': '까다로움, 날카로움', 'details': '아이를 좋아함'}"

json.loads(test.replace('\'','\"'))

{'name': '부장님',
 'gender': '여',
 'relationship': '부장님',
 'personality': '까다로움, 날카로움',
 'details': '아이를 좋아함'}

In [2]:
import pandas as pd 

data0 = None
data = pd.DataFrame(
    {
        "A": [1, 2, 3],
        "B": [1, 2, 3],
        "C": [1, 2, 3],
    }
)
data

,A,B,C
0,1,1,1
1,2,2,2
2,3,3,3


In [3]:
pd.DataFrame(
    columns=["category","start_date","end_date","content","status"]
)

,category,start_date,end_date,content,status


/opt/anaconda3/envs/personaai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1721911933.032531 4981914 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721911933.037773 4981914 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721911933.039156 4981914 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [9]:
import asyncio
import time

from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{input}에 대해 한국어로 5줄로 설명해줘")
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

output = chain.astream({"input": "python"})
async for s in output:
    print(s, end="", flush=True)

I0000 00:00:1722316223.575182 5325681 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722316223.576255 5325681 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


파이썬은 배우기 쉽고 강력한 프로그래밍 언어입니다. 
웹 개발, 데이터 분석, 인공지능 등 다양한 분야에서 사용됩니다. 
간결하고 읽기 쉬운 문법을 가지고 있어 초보자도 쉽게 시작할 수 있습니다. 
다양한 라이브러리와 프레임워크를 제공하여 개발 속도를 높여줍니다. 
전 세계적으로 많은 개발자가 사용하고 있으며, 활발한 커뮤니티를 가지고 있습니다. 


In [6]:
import os
import json 
import requests

def validate_google_api_key():
    """Google API Key 유효성 검사하는 함수"""
    key_name = "GOOGLE_API_KEY"
    if key_name not in os.environ:
        return f"{key_name} 정보가 없습니다. 환경변수를 확인해주세요."
    
    result = requests.post(
        url= "https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent",
        data=b'{"contents":[{"parts":[{"text":""}]}]}',
        headers={
            "Content-Type": "application/json",
            "x-goog-api-key": os.getenv(key_name)
        }
    )

    if result.status_code != 200:
        return json.loads(result.content)
    
    return "[SUCCESS]" # looger 대체

validate_google_api_key()

# API 틀릴 때 
# {'error': {
#         'code': 400,
#         'message': 'API key not valid. Please pass a valid API key.',
#         'status': 'INVALID_ARGUMENT',
#         'details': [
#             {
#                 '@type': 'type.googleapis.com/google.rpc.ErrorInfo',
#                 'reason': 'API_KEY_INVALID',
#                 'domain': 'googleapis.com',
#                 'metadata': {'service': 'generativelanguage.googleapis.com'}
#             }
#         ]
#     }
# }

{'error': {'code': 400,
  'message': 'API key not valid. Please pass a valid API key.',
  'status': 'INVALID_ARGUMENT',
  'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo',
    'reason': 'API_KEY_INVALID',
    'domain': 'googleapis.com',
    'metadata': {'service': 'generativelanguage.googleapis.com'}}]}}

In [1]:
def read_prompt(filepath:str) -> str:
    """프롬프트 파일을 읽고 텍스트로 반환하는 함수

    Args:
        filepath (str): markdown 파일 경로

    Returns:
        str: markdown 파일에서 추출된 텍스트
    """
    file = Path(filepath)
    
    if not file.is_file():
        file_text = f"[ERROR] 파일 경로를 찾을 수 없습니다.(INPUT PATH: {filepath})"
    else:
        file_text = file.read_text(encoding="utf-8")

    return file_text

In [5]:
import os 
from pathlib import Path
from dotenv import load_dotenv 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory

class GeminiChain:
    def __init__(
        self,
        user_info=None,
        character_info=None,
        chat_info=None,
        chat_logs=None
    ) -> None:
        
        self.inputs = self._get_inputs(user_info, character_info, chat_info, chat_logs)
        self.memory = ConversationBufferMemory(
            return_messages=True, 
            memory_key="chat_history"
        )
        self.chain = self._make_chain()

    def _get_inputs(self, user_info, character_info, chat_info, chat_logs):
        inputs = {
            "user_info": user_info,
            "character_info" : character_info,
            "chat_info": chat_info,
            "chat_history": self._get_chat_logs(chat_logs)
        }

        return inputs

    def _get_chat_logs(self, chat_logs):
        if not chat_logs:
            return []

        chat_history = []
        for log in chat_logs:
            if log["role"] == "user":
                chat = HumanMessage(log["contents"])
            else:
                chat = AIMessage(log["contents"])
            
        return chat_history.append(chat)
    
    def _make_chain(self):
        # 프롬프트 설정
        template = read_prompt("./static/templates/Demo.prompt")
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", template),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
            ]
        )

        # 모델 설정
        model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

        # 출력 파서 설정
        output_parser = StrOutputParser()

        # 체인 만들기
        self.chain = prompt | model | output_parser


    def _save_memory(self, input, output):
            self.memory.save_context(
            {"inputs": input},
            {"output": output}
        )

    async def astream(self, input):
        self.inputs["input"] = input

        output = ""
        result = self.chain.astream(self.inputs)
        async for token in result:
            output += token 
            # 소켓 통신 코드 
            print(token, end="", flush=True)
        
        # 메모리에 저장
        self._save_memory(input, output)
        
        return output

In [3]:
chain = GeminiChain()

I0000 00:00:1722056545.034850 5325681 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1722056545.044810 5325681 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722056545.046802 5325681 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [45]:
from langchain_core.messages import AIMessage,HumanMessage

# 날짜(log_create_at) 내림차순
logs = [
    {"role":"user", "contents":"hello"},
    {"role":"character", "contents":"Hi"},
    {"role":"user", "contents":"I'm so happy"},
]

chat_history = []
for log in logs:
    if log["role"] == "user":
        chat = HumanMessage(log["contents"])
    else:
        chat = AIMessage(log["contents"])
    
    chat_history.append(chat)

chat_history

[HumanMessage(content='hello'),
 AIMessage(content='Hi'),
 HumanMessage(content="I'm so happy")]

# MongoDB

atlas로 관리하기: [MongoDB Atlas 평생 무료로 사용해보기](https://it-creamstory.tistory.com/entry/MongoDB-Atlas-%ED%8F%89%EC%83%9D-%EB%AC%B4%EB%A3%8C%EB%A1%9C-%EC%82%AC%EC%9A%A9%ED%95%B4%EB%B3%B4%EA%B8%B0)

In [9]:
# MongoDB 설치 후 사용 버전
import os 
import urllib.parse
from pymongo import MongoClient

# 사용자 정보
username = urllib.parse.quote_plus(os.environ["MONGODB_USERNAME"])
password = urllib.parse.quote_plus(os.environ["MONGODB_PASSWORD"])

# MongoDB 호스트 및 포트 정보
host = "172.16.2.10"
port = 27017

# MongoDB URI
mongo_uri = f"mongodb://{username}:{password}@{host}:{port}"

# MongoClient 생성
client = MongoClient(mongo_uri)

# 데이터베이스 이름
database_name = "emozis"

# 데이터베이스 선택
db = client[database_name]

# 연결 확인
print(f"데이터베이스 '{database_name}'에 연결되었습니다.")

데이터베이스 'emozis'에 연결되었습니다.


## 실습

In [2]:
# Atlas Version
import os 
import urllib.parse
from pymongo import MongoClient

# 사용자 정보
username = urllib.parse.quote_plus(os.environ["MONGODB_USERNAME"])
password = urllib.parse.quote_plus(os.environ["MONGODB_PASSWORD"])

# MongoDB 호스트 및 포트 정보
host = "cluster0.urikuvt.mongodb.net"

# MongoDB URI
mongo_uri = f"mongodb+srv://{username}:{password}@{host}"

# MongoClient 생성
client = MongoClient(mongo_uri)

# 데이터베이스 이름
database_name = "emozis"

# 데이터베이스 선택
db = client[database_name]

# 연결 확인
print(f"데이터베이스 '{database_name}'에 연결되었습니다.")

데이터베이스 'emozis'에 연결되었습니다.


In [3]:
# client의 모든 데이터베이스 조회
client.list_database_names()

['sample_mflix', 'admin', 'local']

In [4]:
# 컬렉션 생성
collection_name = "project"
db.create_collection(collection_name)   # 생성
# db.drop_collection(collection_name)   # 삭제

Collection(Database(MongoClient(host=['ac-feuml5i-shard-00-01.urikuvt.mongodb.net:27017', 'ac-feuml5i-shard-00-02.urikuvt.mongodb.net:27017', 'ac-feuml5i-shard-00-00.urikuvt.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-t7l5oj-shard-0', tls=True), 'emozis'), 'project')

In [6]:
# 데이터베이스의 모든 컬렉션 조회
db.list_collection_names()

['project']

In [7]:
collection = db["project"]
collection

Collection(Database(MongoClient(host=['ac-feuml5i-shard-00-01.urikuvt.mongodb.net:27017', 'ac-feuml5i-shard-00-02.urikuvt.mongodb.net:27017', 'ac-feuml5i-shard-00-00.urikuvt.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-t7l5oj-shard-0', tls=True), 'emozis'), 'project')

In [8]:
# 문서 추가
# 단일 문서 삽입
document = {"name": "Alice", "age": 30, "city": "New York"}
insert_result = collection.insert_one(document)
print(f"Inserted document ID: {insert_result.inserted_id}")

Inserted document ID: 66aba359133700f1dc2abdc4


In [9]:
# 다중 문서 삽입
documents = [
    {"name": "Bob", "age": 25, "city": "Los Angeles"},
    {"name": "Charlie", "age": 35, "city": "Chicago"},
]
insert_many_result = collection.insert_many(documents)
print(f"Inserted document IDs: {insert_many_result.inserted_ids}")

Inserted document IDs: [ObjectId('66aba35b133700f1dc2abdc5'), ObjectId('66aba35b133700f1dc2abdc6')]


In [10]:
# 모든 문서 조회
all_documents = collection.find()
for doc in all_documents:
    print(doc)

{'_id': ObjectId('66aba359133700f1dc2abdc4'), 'name': 'Alice', 'age': 30, 'city': 'New York'}
{'_id': ObjectId('66aba35b133700f1dc2abdc5'), 'name': 'Bob', 'age': 25, 'city': 'Los Angeles'}
{'_id': ObjectId('66aba35b133700f1dc2abdc6'), 'name': 'Charlie', 'age': 35, 'city': 'Chicago'}


In [11]:
# 특정 조건의 문서 조회
query = {"age": {"$gt": 30}}  # 나이가 30보다 큰 문서 조회
filtered_documents = collection.find(query)
for doc in filtered_documents:
    print(doc)

{'_id': ObjectId('66aba35b133700f1dc2abdc6'), 'name': 'Charlie', 'age': 35, 'city': 'Chicago'}


In [12]:
# 단일 문서 업데이트
query = {"name": "Alice"}
new_values = {"$set": {"age": 31}}
update_result = collection.update_one(query, new_values)
print(f"Matched documents: {update_result.matched_count}, Updated documents: {update_result.modified_count}")

Matched documents: 1, Updated documents: 1


In [13]:
# 다중 문서 업데이트
query = {"city": "Los Angeles"}
new_values = {"$set": {"city": "San Francisco"}}
update_many_result = collection.update_many(query, new_values)
print(f"Matched documents: {update_many_result.matched_count}, Updated documents: {update_many_result.modified_count}")

Matched documents: 1, Updated documents: 1


In [14]:
# 단일 문서 삭제
query = {"name": "Alice"}
delete_result = collection.delete_one(query)
print(f"Deleted documents: {delete_result.deleted_count}")

Deleted documents: 1


In [15]:
# 다중 문서 삭제
query = {"city": "San Francisco"}
delete_many_result = collection.delete_many(query)
print(f"Deleted documents: {delete_many_result.deleted_count}")

Deleted documents: 1


In [16]:
# 모든 문서 조회
all_documents = collection.find()
for doc in all_documents:
    print(doc)

{'_id': ObjectId('66aba35b133700f1dc2abdc6'), 'name': 'Charlie', 'age': 35, 'city': 'Chicago'}


In [18]:
# 모든 문서 삭제
collection.delete_many({})

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000000c0'), 'opTime': {'ts': Timestamp(1722524581, 1), 't': 192}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1722524581, 2), 'signature': {'hash': b'\x1e 6\xc1P;\x96\x8d\xeele\x14\xe6\xf217\xf3a^7', 'keyId': 7340700305501192197}}, 'operationTime': Timestamp(1722524581, 1)}, acknowledged=True)

## 데이터 삽입

In [19]:
import pandas as pd 
import json

with open("../data_frame.json", 'r') as file:
    data = json.load(file)

data[:2]

[{'category': 'AI',
  'start_date': '2024-07-10T00:00:00.000',
  'end_date': '2024-08-05T00:00:00.000',
  'content': '🪄 프롬프트 업데이트\n\n✅ 시어머니 \n✅ 인사이드아웃\n✅ 여사친\n✅ 남사친',
  'status': '🔴 진행중'},
 {'category': 'AI',
  'start_date': '2024-07-10T00:00:00.000',
  'end_date': '2024-08-05T00:00:00.000',
  'content': '🪄 Demo 페이지 구현\n\n✅ 프롬프트 선택 가능, 직접 입력 가능\n✅ 챗봇 구현 (오류 해결)\n➕ 503 error 자동 API 재요청 코드로 전환\n➕ 충돌 방지를 위해 프롬프트 편집 폴더를 따로 분리',
  'status': '🟢 완료'}]

In [20]:
collection.insert_many(data)

InsertManyResult([ObjectId('66aba3fc133700f1dc2abdc7'), ObjectId('66aba3fc133700f1dc2abdc8'), ObjectId('66aba3fc133700f1dc2abdc9'), ObjectId('66aba3fc133700f1dc2abdca'), ObjectId('66aba3fc133700f1dc2abdcb'), ObjectId('66aba3fc133700f1dc2abdcc'), ObjectId('66aba3fc133700f1dc2abdcd'), ObjectId('66aba3fc133700f1dc2abdce'), ObjectId('66aba3fc133700f1dc2abdcf'), ObjectId('66aba3fc133700f1dc2abdd0'), ObjectId('66aba3fc133700f1dc2abdd1'), ObjectId('66aba3fc133700f1dc2abdd2'), ObjectId('66aba3fc133700f1dc2abdd3'), ObjectId('66aba3fc133700f1dc2abdd4'), ObjectId('66aba3fc133700f1dc2abdd5'), ObjectId('66aba3fc133700f1dc2abdd6'), ObjectId('66aba3fc133700f1dc2abdd7')], acknowledged=True)